<a href="https://colab.research.google.com/github/Yoon-juhan/naverNewsCrawling/blob/main/%EB%89%B4%EC%8A%A4_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 뉴스 크롤링

In [1]:
pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 448.3/448.3 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.2 MB/s eta 0:00:00


In [2]:
pip install webdriver-manager

In [3]:
!apt-get update
!apt install chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [46.6 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,419 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.8 kB]
Hit:13 https://ppa.launchpadc

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
import datetime
from pytz import timezone
KST = timezone('Asia/Seoul')

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)
# browser.implicitly_wait(1)

## 기사 링크 크롤링

In [5]:
# 기사 링크 크롤링
# 카테고리당 100개 = 5페이지

six_url_list = []   # url 저장 리스트
entertainment_url_list = []
sports_url_list = []

category_list = []
categoty_name = ["정치", "경제", "사회", "생활/문화", "세계", "IT/과학"]

def getSixLink():    # 정치, 경제, 사회, 생활/문화, 세계, IT/과학

    for category in range(6):     # 6
        a_list = []
        for page in range(1, 11):  # 1, 6
            url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
            browser.get(url)

            time.sleep(0.5)

            soup = BeautifulSoup(browser.page_source, "html.parser")
            a_list.extend(soup.select(".type06_headline dt+dt a"))
            a_list.extend(soup.select(".type06 dt+dt a"))

            print(f"{categoty_name[category]} {page} 페이지")

        for a in a_list:
            six_url_list.append(a["href"])
            category_list.append(categoty_name[category])

def getEntertainmentLink():   # 연예
    # today = str(datetime.datetime.now(KST))[:11]  # 서울 기준 시간
    a_list = []
    today = datetime.date.today()

    for page in range(1, 9):  # 1, 5
        url = f'https://entertain.naver.com/now#sid=106&date={today}&page={page}'
        browser.get(url)

        time.sleep(0.5)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        a_list.extend(soup.select(".news_lst li>a"))


        print(f"연예 {page} 페이지")

    for a in a_list:
        entertainment_url_list.append("https://entertain.naver.com" + a["href"])
        category_list.append("연예")

def getSportsLink():    # 스포츠  (페이지마다 개수가 달라서 6페이지를 이동)
    # today = str(datetime.datetime.now(KST))[:11].replace('-', '')  # 서울 기준 시간
    a_list = []
    today = str(datetime.date.today()).replace('-', '')

    for page in range(1, 2):  # 1, 7
        url = f'https://sports.news.naver.com/general/news/index?isphoto=N&type=latest&date={today}&page={page}'
        browser.get(url)

        time.sleep(0.5)

        soup = BeautifulSoup(browser.page_source, "html.parser")
        a_list.extend(soup.select(".news_list li>a"))

        print(f"스포츠 {page} 페이지")

    for i in range(len(a_list)):
        if i == 100:  # 100개 링크 추가했으면 멈추기
            break
        sports_url_list.append("https://sports.news.naver.com/news" + re.search('\?.+', a_list[i]["href"]).group())
        category_list.append("스포츠")

getSixLink()
getEntertainmentLink()
# getSportsLink()


정치 1 페이지
정치 2 페이지
정치 3 페이지
정치 4 페이지
정치 5 페이지
정치 6 페이지
정치 7 페이지
정치 8 페이지
정치 9 페이지
정치 10 페이지
경제 1 페이지
경제 2 페이지
경제 3 페이지
경제 4 페이지
경제 5 페이지
경제 6 페이지
경제 7 페이지
경제 8 페이지
경제 9 페이지
경제 10 페이지
사회 1 페이지
사회 2 페이지
사회 3 페이지
사회 4 페이지
사회 5 페이지
사회 6 페이지
사회 7 페이지
사회 8 페이지
사회 9 페이지
사회 10 페이지
생활/문화 1 페이지
생활/문화 2 페이지
생활/문화 3 페이지
생활/문화 4 페이지
생활/문화 5 페이지
생활/문화 6 페이지
생활/문화 7 페이지
생활/문화 8 페이지
생활/문화 9 페이지
생활/문화 10 페이지
세계 1 페이지
세계 2 페이지
세계 3 페이지
세계 4 페이지
세계 5 페이지
세계 6 페이지
세계 7 페이지
세계 8 페이지
세계 9 페이지
세계 10 페이지
IT/과학 1 페이지
IT/과학 2 페이지
IT/과학 3 페이지
IT/과학 4 페이지
IT/과학 5 페이지
IT/과학 6 페이지
IT/과학 7 페이지
IT/과학 8 페이지
IT/과학 9 페이지
IT/과학 10 페이지
연예 1 페이지
연예 2 페이지
연예 3 페이지
연예 4 페이지
연예 5 페이지
연예 6 페이지
연예 7 페이지
연예 8 페이지


In [7]:
print("6개 카테고리 수집 개수 :", len(six_url_list))
print("6개 카테고리 수집 개수 :", len(set(six_url_list)))
print()
print("연예 카테고리 수집 개수 :", len(entertainment_url_list))
print("연예 카테고리 수집 개수 :", len(set(entertainment_url_list)))
print()
print("스포츠 카테고리 수집 개수 :", len(sports_url_list))
print("스포츠 카테고리 수집 개수 :", len(set(sports_url_list)))
print()
print("전체 카테고리 수집 개수 : ", len(category_list))

6개 카테고리 수집 개수 : 1200
6개 카테고리 수집 개수 : 1176

연예 카테고리 수집 개수 : 199
연예 카테고리 수집 개수 : 199

스포츠 카테고리 수집 개수 : 0
스포츠 카테고리 수집 개수 : 0

전체 카테고리 수집 개수 :  1399


## 제목, 본문, 날짜 크롤링

In [8]:
# 수집한 태그 속에서 내용만 뽑아서 담는다.

def clean(article):   # 전처리
    article = re.sub('\w+ 기자','',article)
    article = re.sub('\w{2,4} 온라인 기자','',article)
    article = re.sub('\w{2,4}기자','',article)
    article = re.sub('사진','',article)
    article = re.sub('포토','',article)
    # article = re.sub('\S+@[a-z.]+','',article)

    article = re.sub('\n','',article)
    article = re.sub('\t','',article)
    article = re.sub('\u200b','',article)
    article = re.sub('\xa0','',article)
    article = re.sub('([a-zA-Z])','',article)
    article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
    article = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',article)
    article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)


    return article


In [9]:
# 수집한 링크의 제목, 본문, 날짜 크롤링

title_list = []     # 제목
content_list = []   # 본문
date_list = []      # 날짜

def getSixContent(url_list):   # 정치, 경제, 사회, 생활/문화, 세계, IT/과학
    title = []
    content = []
    date = []

    for url in url_list:
        browser.get(url)

        time.sleep(0.5)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select("#title_area span"))                 # 제목

        c = soup.find_all(attrs={"id" : "dic_area"})                  # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

        while c[0].find(attrs={"class" : "vod_player_wrap"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "vod_player_wrap"}).decompose()  # 본문 영상에 있는 글자 없애기

        content.extend(c)

        date.extend(soup.select("._ARTICLE_DATE_TIME"))               # 날짜

    for t in title:
        title_list.append(clean(t.text))

    for c in content:
        content_list.append(clean(c.text))

    for d in date:
        date_list.append(d.text)


def getEntertainmentContent(url_list):    # 연예
    title = []
    content = []
    date = []

    for url in url_list:
        browser.get(url)

        time.sleep(0.5)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select(".end_tit"))                 # 제목

        c = soup.find_all(attrs={"id" : "articeBody"})                # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기
          if c[0].find(attrs={"class" : "caption"}):                  # 이미지 설명 없애기
            c[0].find(attrs={"class" : "caption"}).decompose()

        while c[0].find(attrs={"class" : "video_area"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "video_area"}).decompose()  # 본문 영상에 있는 글자 없애기

        while c[0].find(attrs={"name" : "iframe"}):
           c[0].find(attrs={"name" : "iframe"}).decompose()

        content.extend(c)

        date.extend(soup.select_one(".author em"))               # 날짜

    for t in title:
      title_list.append(clean(t.text))

    for c in content:
      content_list.append(clean(c.text))

    for d in date:
      date_list.append(d.text)


def getSportsContent(url_list):   # 스포츠
    title = []
    content = []
    date = []

    for url in url_list:

        browser.get(url)
        print(url)
        time.sleep(0.5)

        soup = BeautifulSoup(browser.page_source, "html.parser")

        title.extend(soup.select(".news_headline .title"))                 # 제목

        c = soup.find_all(attrs={"class" : "news_end"})                # 본문

        while c[0].find(attrs={"class" : "end_photo_org"}):           # 이미지 있는 만큼
          c[0].find(attrs={"class" : "end_photo_org"}).decompose()    # 본문 이미지에 있는 글자 없애기

        while c[0].find(attrs={"class" : "image"}):
           c[0].find(attrs={"class" : "image"}).decompose()

        while c[0].find(attrs={"class" : "vod_area"}):         # 영상 있는 만큼
          c[0].find(attrs={"class" : "vod_area"}).decompose()  # 본문 영상에 있는 글자 없애기

        c[0].find(attrs={"class" : "source"}).decompose()
        c[0].find(attrs={"class" : "byline"}).decompose()
        c[0].find(attrs={"class" : "reporter_area"}).decompose()
        c[0].find(attrs={"class" : "copyright"}).decompose()
        c[0].find(attrs={"class" : "categorize"}).decompose()
        c[0].find(attrs={"class" : "promotion"}).decompose()

        content.extend(c)

        date.extend(soup.select_one(".info span"))               # 날짜

    for t in title:
      title_list.append(clean(t.text))

    for c in content:
      content_list.append(clean(c.text))

    for d in date:
      date_list.append(d.text)

getSixContent(six_url_list)
getEntertainmentContent(entertainment_url_list)
# getSportsContent(sports_url_list)


In [10]:
print(len(title_list))
print(len(content_list))
print(len(date_list))

1399
1399
1399


## 데이터 프레임 생성

In [ ]:
# 데이터 프레임으로 변환
all_url_list = six_url_list + entertainment_url_list + sports_url_list

article_df = pd.DataFrame({"category" : category_list,
                           "date" : date_list,
                           "title" : title_list,
                           "content" : content_list,
                           "url" : all_url_list})

article_df

In [21]:
# article_df.to_csv("C:\\Users\\82106\\OneDrive\\바탕 화면\\article.csv", encoding='utf-8-sig')

# 전처리

**KoNLPy**
- 한국어 정보처리를 위한 파이썬 패키지
- https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class


In [30]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 33.4 MB/s eta 0:00:00


In [104]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import matplotlib.pyplot as plt


# CountVectorizer : 텍스트 빈도만을 카운트해서 벡터화
# TfidfVectorizer : 자주 나오는 단어에 높은 가중치, 모든 문서에서 자주 나오는 단어에 패널티

In [121]:
def getNouns():
    okt = Okt()
    nouns_list = []   # 명사 리스트

    for content in article_df["content"]:
        nouns_list.append(okt.nouns(content))     # 명사 추출, 리스트 반환

    article_df["nouns"] = nouns_list   # 데이터 프레임에 추가


getNouns()

In [38]:
# 명사까지 추출한 파일 불러오기, 1400개 기사

test_df = pd.read_csv("https://raw.githubusercontent.com/Yoon-juhan/naverNewsCrawling/main/article.csv")
test_df['nouns'] = test_df['nouns'].apply(lambda x: eval(x))        # 명사 열을 다시 리스트 형식으로 변환

In [93]:
# 영어기사 삭제

english_article_index = test_df[test_df['nouns'] == '[]'].index
test_df.drop(english_article_index, inplace=True)

In [146]:
def getVector():    # 카테고리 별로 돌리기
    text = [" ".join(noun) for noun in test_df["nouns"]]  # 명사 열을 하나의 리스트에 담는다.

    tfidf_vectorizer = TfidfVectorizer(min_df = 3, ngram_range=(1, 5))
    tfidf_vectorizer.fit(text)
    vector = tfidf_vectorizer.transform(text).toarray()  # vector list 반환
    vector = np.array(vector)

    return vector

vector = getVector()

['홍준표 대구시 곧 정계 빅뱅 것 대처 경고 홍 시장 자신 나 우리 당 국회의원 협 위원장 극히 일부 제외 채권 채무 대선 후보 경선 당시 대부분 국회의원 협 위원장 다른 후보 진영 때문 설명 지난번 내년 총선 후 정치 지형 아래 다시 시작 고 일각 탈당 모양 나 이 당 여 년 온 본류 지류 본류 지난번 황교안 대표 이번 탈당 선 었습 본류 제 길 이탈 대홍수 탄핵 와중 당 내 탈당 일 절대 내용 글 여 년 정치 당 권력 구도 정치 수 것 그 기반 권력자 국민 때문 라며 곧 정치 지형 것 지적 자칭 친 윤계 그룹 정권 출범 초기 대통령 등 호가호위 그 결과 당 위계 질서 선후배 중진 이 눈치 당내 무질서 오늘 당 중심 세력 현상 초래 것 라면 당 위기 원인 분석 홍 시장 총선 지면 식물 정부 사람 나 전 혼란 예견 난 피해 하방 곧 나라 엑소더스 당내 자칭 친윤 경고', '일 군사 협력 핵전쟁 발발 차 대전 겁박 북한 미사일 공업 절 로 지정 북한 실시 대륙간탄도미사일 화성 형 시험 발사 사실 성공 것 기념 위 목적 북한 미사일 공업 절 제정 군사 정찰위성 재 발사 등 도발 수 우려 제기 북한 재 발사 성공 위해 기술 보완 어가 등 당분 움직임 것 분석 힘 조선 중앙 통신 북한 최고 인민 회의 상임 위원회 상무 회의 미사일 공업 절 제정 관 문제 등 안건 상 정해 전원 찬성 채택 보도 통신 세계 핵 강국 최강 대륙간탄도미사일 보유 국 위용 만천하 며 우리 국방 발전 여정 특기 사변 이 날 평가 전문가 북한 미사일 공업 절 제정 핵 미사일 기술 점 내외 과시 의도 해석 다홍 민 통일 연구원 선임 위원 국민일보 통화 미사일 공업 절 화성 형 발사 날로 정 때문 미국 본토 핵 타격 수 국가 의미 고 해석 다홍 선임 위원 외적 대미 억제 과시 목적 대내 핵 미사일 고도화 업적 각인 환기 것 부연 북한 미사일 공업 절 기념 위해 정찰위성 발사 등 무력 도발 것 관측 북한 지난 두 차례 정찰위성 발사 중 정찰위성 발사 를 예고 실행 무진 북한 대학원 총장 예고 정찰위성

In [147]:
vector

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# 군집화

**DBSCAN**
- https://bcho.tistory.com/1205


In [148]:
from sklearn.cluster import DBSCAN

# DBSCAN : 밀도 기반 클러스터링
# 점 p에서 부터 거리 e (epsilon)내에 점이 m(minPts) 개 있으면 하나의 군집으로 인식

In [149]:
model = DBSCAN(eps=0.1, min_samples=1, metric='cosine')
result = model.fit_predict(vector)
result

array([   0,    1,    2, ..., 1139, 1185, 1123])

In [150]:
test_df['cluster_number'] = result

In [ ]:
# 군집 개수를 계산하고 내림차순으로 정렬
cluster_counts_df = test_df['cluster_number'].value_counts().reset_index()
cluster_counts_df.columns = ['cluster_number', 'cluster_count']
# cluster_counts_df = cluster_counts_df.sort_values(by='cluster_count', ascending=False)

cluster_counts_df

In [ ]:
test_df[test_df['cluster_number'] == cluster_counts_df['cluster_number'][4]]

In [ ]:
for n in set(result):   # 군집 결과
    print(f"cluster_number : {n}")   # 군집 번호
    temp_df = test_df[test_df["cluster_number"] == n]
    for title in temp_df["title"]:
        print(title)
    print()